# Reflexion Prompting with OpenAI

**Reflexion** = Self-correction through reflection.

The model:
1. Tries to solve a problem
2. Gets feedback (or self-assesses)
3. Reflects: "What went wrong? How can I improve?"
4. Generates a better strategy
5. Retries with improved reasoning

Extremely effective for math, coding, logic puzzles, and complex reasoning.

In [ ]:
!pip install openai python-dotenv --quiet

In [ ]:
import openai
import os
from dotenv import load_dotenv
import time

load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")
if not openai.api_key:
    openai.api_key = input("Paste your OpenAI API key: ").strip()

print("OpenAI ready! Using gpt-4o-mini (excellent at reflection)")

## Reflexion Engine (Self-Improving Agent)

In [ ]:
def reflexion_solve(problem: str, max_reflections: int = 4):
    reflections = []
    attempts = []

    print(f"Problem:\n{problem}\n")
    print("Starting Reflexion Agent...\n")
    time.sleep(1)

    for attempt in range(1, max_reflections + 2):
        print(f"ATTEMPT {attempt}")
        print("-" * 50)

        # Build reflection history
        reflection_text = ""
        if reflections:
            reflection_text = "Previous reflections on mistakes:\n" + "\n".join(reflections[-3:]) + "\n\nUse these insights to avoid repeating errors."

        prompt = f"""
{reflection_text}

Solve this problem step by step:
{problem}

Think carefully. Then give your final answer in this format:
<FINAL_ANSWER>\n[your answer here]\n</FINAL_ANSWER>
"""

        response = openai.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": "You are a self-reflecting reasoning agent. Learn from past mistakes."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.6 if attempt == 1 else 0.4,  # More creative first, then focused
            max_tokens=800
        )

        reply = response.choices[0].message.content
        print(reply)
        attempts.append(reply)

        # Try to extract final answer
        if "<FINAL_ANSWER>" in reply:
            answer = reply.split("<FINAL_ANSWER>")[1].split("</FINAL_ANSWER>")[0].strip()
            print(f"\nSUCCESS! Final Answer:\n{answer}\n")
            return answer, attempts, reflections

        # If no final answer → trigger reflection
        if attempt <= max_reflections:
            print("\nNo valid answer → Triggering self-reflection...\n")
            reflect_prompt = f"""
The previous attempt failed to produce a correct or complete answer.

Here was the attempt:
{reply}

Reflect deeply:
- What went wrong?
- What assumptions were incorrect?
- What step was missed or misunderstood?
- How should you reason differently next time?

Be honest and specific.
"""

            reflect_response = openai.chat.completions.create(
                model="gpt-4o-mini",
                messages=[{"role": "user", "content": reflect_prompt}],
                temperature=0.7
            )

            reflection = reflect_response.choices[0].message.content.strip()
            reflections.append(f"Reflection {len(reflections)+1}: {reflection}")
            print(f"Reflection:\n{reflection}\n")
            time.sleep(1)

    print("Max reflections reached. Best effort shown above.")
    return None, attempts, reflections

## Try Reflexion on Real Problems!

In [ ]:
# Example 1: Classic hard logic/math problem
problem1 = """
You have 3 light bulbs in a room and 3 switches outside. Each switch controls one bulb.
One switch turns it on/off, one does nothing, one turns it on only when the door is closed.
You can enter the room only once.

How do you determine which switch does what?
"""

# Example 2: Tricky math riddle
problem2 = """
A lily pad doubles in size every day. On day 30, it covers the entire pond.
On what day did it cover half the pond?
"""

# Example 3: Reasoning puzzle
problem3 = """
Einstein's riddle: There are 5 houses in a row, each with a different color, owner of different nationality, drinking different beverage, smoking different brand, and having different pet.
Who owns the fish?
(Provide the famous full riddle if needed — but the model knows it!)
"""

print("Choose a problem:\n")
print("1. Light bulb puzzle")
print("2. Lily pad math riddle")
print("3. Einstein's fish riddle")
print("4. Custom problem")

choice = input("\nEnter 1–4: ").strip()

if choice == "1":
    problem = problem1
elif choice == "2":
    problem = problem2
elif choice == "3":
    problem = "Solve Einstein's riddle: Who owns the fish? Include all clues."
elif choice == "4":
    problem = input("\nPaste your own problem:\n")
else:
    problem = problem1

# Run Reflexion!
final_answer, attempts, reflections = reflexion_solve(problem, max_reflections=4)

You now have a fully working **Reflexion Agent** — one of the most advanced prompting techniques available in 2025.

Use it for:
- Hard math/logic problems
- Coding challenges
- Multi-step reasoning
- Any task where first attempts often fail

Next-level: Combine with **Self-Consistency** or **Tree of Thought** for even higher accuracy!

Want **Plan-and-Solve**, **Skeleton-of-Thought**, or **CriticGPT-style** next? Just ask!